In [1]:
import sys
import os
import pandas as pd
import numpy as np
import scipy
import gget
import sklearn.preprocessing 
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import fcluster
from importlib import reload
import matplotlib.pyplot as plt
from matplotlib import patches
import seaborn as sns
import networkx as nx
import xgi

source_path = os.path.abspath("../source/")
sys.path.append(source_path)
import utils as ut
import plotting as plt2
import hypercore as hc
import matrix as matrix
import centrality as central

In [7]:
# load the population data
fpath = "/scratch/indikar_root/indikar1/shared_data/higher_order/population_pore_c/chr1_genes_incidence.parquet"

df = pd.read_parquet(fpath)
print(f"{df.shape=}")
df.head()

df.shape=(3510, 83636)


read_code,0,1,2,3,4,5,6,7,8,9,...,83626,83627,83628,83629,83630,83631,83632,83633,83634,83635
gene_name,,,,,,,,,,,,,,,,,,,,,
1110002O04Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1600012P17Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1700001G17Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1700003I22Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1700006P03Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
reload(central)

nodes = []

measures = [
    'log-exp', 
]

for func in measures:
    ncentrality, _ = central.nonlinear_eigenvector_centrality(df.to_numpy(), 
                                                        function=func, 
                                                        maxiter=1000)
    
    ncentrality = ut.min_max(ncentrality)
    nodes.append(pd.DataFrame({func : ncentrality,}, index=df.index))
nodes = pd.concat(nodes, axis=1).reset_index()
nodes = nodes.sort_values(by='log-exp', ascending=False)

nodes.head()

,gene_name,log-exp
3313,Sp140l2,1.000000
3019,Pard3b,0.935518
519,Dpp10,0.930035
3128,Rabgap1l,0.915133
2641,Hmcn1,0.902013


In [21]:
n_genes = 50

genes = nodes.head(n_genes)['gene_name'].to_list()
print(genes)

database = 'ontology'
result = gget.enrichr(genes, database=database)

result.head(5)

Mon Jun 10 15:51:56 2024 INFO Performing Enichr analysis using database GO_Biological_Process_2021. 
    Please note that there might a more appropriate database for your application. 
    Go to https://maayanlab.cloud/Enrichr/#libraries for a full list of supported databases.
    


['Sp140l2', 'Pard3b', 'Dpp10', 'Rabgap1l', 'Hmcn1', 'Nckap5', 'Erbb4', 'Thsd7b', 'Dis3l2', 'Adgrb3', 'Spag16', 'Esrrg', 'Sntg1', 'Ush2a', 'Gm6264', 'Gm20631', 'Gm36975', 'Cntnap5a', 'Tnr', 'A830018L16Rik', 'Smyd3', 'Xkr4', 'Gm28376', '6030407O03Rik', 'Cntnap5b', 'Jph1', 'Kcnq5', 'Sp140l1', 'Dnm3', 'Pkhd1', 'Agap1', 'Kif26b', 'Gm28375', 'Ankrd44', 'Aff3', 'Kcnt2', 'Astn1', 'Rims1', 'Cacna1e', 'Brinp3', 'Rgs7', 'Dst', 'Kcnb2', 'Gm29183', 'Gm29536', 'Cpa6', 'Pid1', 'Kcnh1', 'Plcl1', 'Prex2']


,rank,path_name,p_val,z_score,combined_score,overlapping_genes,adj_p_val,database
0,1,establishment of centrosome localization (GO:0...,0.000128,166.208333,1490.324070,"[PKHD1, PARD3B]",0.042364,GO_Biological_Process_2021
1,2,potassium ion transmembrane transport (GO:0071...,0.000402,12.763285,99.811947,"[KCNT2, KCNB2, KCNQ5, KCNH1]",0.066653,GO_Biological_Process_2021
2,3,myoblast fusion (GO:0007520),0.000630,63.900641,470.944287,"[ADGRB3, KCNH1]",0.069709,GO_Biological_Process_2021
3,4,myotube differentiation (GO:0014902),0.001248,43.708333,292.250475,"[ADGRB3, KCNH1]",0.103567,GO_Biological_Process_2021
4,5,potassium ion transport (GO:0006813),0.003522,10.637046,60.085476,"[KCNB2, KCNQ5, KCNH1]",0.202239,GO_Biological_Process_2021


In [4]:
fpath = "/scratch/indikar_root/indikar1/shared_data/higher_order/rna/expression.parquet"
gdf = pd.read_parquet(fpath)

resolution = 100000

gdf = gdf[gdf['Chromosome'] == "1"]
gdf = gdf[gdf['gene_biotype'] == "protein_coding"]
gdf['bin'] = gdf['midpoint'].apply(lambda x: ut.bin_loci(x, resolution))

edf = gdf.groupby('bin').agg(
    # GENE_COUNT = ('gene_name', 'count'),
    # TF_COUNT = ('is_tf', 'sum'),
    TPM_4DNFI8CSCJWM = ('4DNFI8CSCJWM_TPM', 'mean'), 
    TPM_4DNFICXJQ3PA = ('4DNFICXJQ3PA_TPM', 'mean'), 
    TPM_4DNFI3YYNDKI = ('4DNFI3YYNDKI_TPM', 'mean'), 
    TPM_4DNFIPYGE7JR = ('4DNFIPYGE7JR_TPM', 'mean'), 
    TPM_4DNFIYTCHMIZ = ('4DNFIYTCHMIZ_TPM', 'mean'), 
    TPM_4DNFIC269AEU = ('4DNFIC269AEU_TPM', 'mean'), 
).fillna(0)
edf = np.log1p(edf)

edf = pd.merge(edf.reset_index(), nodes)

print(f"{edf.shape=}")
edf = edf.set_index('bin')
edf.head()

edf.shape=(741, 8)


,TPM_4DNFI8CSCJWM,TPM_4DNFICXJQ3PA,TPM_4DNFI3YYNDKI,TPM_4DNFIPYGE7JR,TPM_4DNFIYTCHMIZ,TPM_4DNFIC269AEU,log-exp
bin,,,,,,,
36.0,0.000000,0.019803,0.000000,0.000000,0.000000,0.019803,0.000531
43.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000517
46.0,0.774727,1.068153,0.875469,0.875469,0.774727,1.068153,0.000521
49.0,2.741807,4.711330,2.598979,2.598979,2.741807,4.711375,0.000523
50.0,1.507405,3.903688,1.565486,1.565486,1.507405,3.903688,0.000527


In [5]:
edf.corr()

,TPM_4DNFI8CSCJWM,TPM_4DNFICXJQ3PA,TPM_4DNFI3YYNDKI,TPM_4DNFIPYGE7JR,TPM_4DNFIYTCHMIZ,TPM_4DNFIC269AEU,log-exp
TPM_4DNFI8CSCJWM,1.000000,0.902150,0.983829,0.983850,0.999999,0.902151,0.173365
TPM_4DNFICXJQ3PA,0.902150,1.000000,0.903172,0.903213,0.902134,1.000000,0.180852
TPM_4DNFI3YYNDKI,0.983829,0.903172,1.000000,0.999999,0.983835,0.903172,0.173524
TPM_4DNFIPYGE7JR,0.983850,0.903213,0.999999,1.000000,0.983856,0.903213,0.173563
TPM_4DNFIYTCHMIZ,0.999999,0.902134,0.983835,0.983856,1.000000,0.902136,0.173338
TPM_4DNFIC269AEU,0.902151,1.000000,0.903172,0.903213,0.902136,1.000000,0.180852
log-exp,0.173365,0.180852,0.173524,0.173563,0.173338,0.180852,1.000000


In [6]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

# Hypergraph Centrality

In [ ]:
reload(central)

nodes = []

measures = [
    'log-exp', 
]

for func in measures:
    ncentrality, _ = central.nonlinear_eigenvector_centrality(df.to_numpy(), 
                                                        function=func, 
                                                        maxiter=1000)
    
    nodes.append(pd.DataFrame({func : ncentrality,}, index=df.index))
    
    
nodes = pd.concat(nodes, axis=1).reset_index()

nodes.head()